In [1]:
import sys
sys.path.append('/home/tnguser/python/')
sys.path.append('/home/tnguser/python/BFE_analysis/')
sys.path.append('/home/tnguser/python/Principal_axes_analysis/')

import numpy as np
import h5py
import os
from FigureRotation_util import *
from Cannon_get_principal_axes import *
from prob_plane_method import *

from astropy import constants as const
from astropy import units as u
from astropy.cosmology import Planck15 as cosmo
from tqdm import tqdm
from os.path import exists 

In [3]:
sim = 'L35n2160TNG'
basePath = '/home/tnguser/sims.TNG/' + sim + '/output'
final_diskyIDs = np.load('/home/tnguser/postprocessing/circularity_study/real_final_diskyIDs.npy')
final_GrNr = []
for subfindID in final_diskyIDs:
    GrNr = il.groupcat.loadSingle(basePath, 99, subhaloID=subfindID)['SubhaloGrNr']
    final_GrNr.append(il.groupcat.loadSingle(basePath, 99, subhaloID=subfindID)['SubhaloGrNr'])

In [4]:
sim = 'L35n2160TNG'
innerVirialLim=0.0
outerVirialLim=0.6

#### Replace w/ new selected GrNrs ######################################
saarthak_catalogue = np.array(final_GrNr)
#########################################################################


#### Set up the paths, load error coefficients ####

startSnap = 75

saveDir = "/home/tnguser/postprocessing/pattern_speeds/"+sim+"/%02d_%02dRvir/"%(innerVirialLim*10,outerVirialLim*10)
loadDir = "/home/tnguser/postprocessing/principal_axes/"+sim+"/%02d_%02dRvir/"%(innerVirialLim*10,outerVirialLim*10)
frameDir = "/home/tnguser/postprocessing/principal_axes/"+sim+"/%02d_%02dRvir/"%(innerVirialLim*10,outerVirialLim*10)
basePath = '/home/tnguser/sims.TNG/'+sim+'/output'

errFitConsts = np.load("/home/tnguser/postprocessing/"+\
                               "empirical_error_fits_full.npy",allow_pickle=True)

In [5]:
snapArr,zArr = snapshot_redshift_corr(basePath)

t_arr = [0]
for snap in range(75,99):
    z0 = zArr[snap==snapArr]
    z1 = zArr[snap+1==snapArr]
    t_arr.append((cosmo.lookback_time(z0).value - cosmo.lookback_time(z1).value)[0])
t_arr = np.array(t_arr)

f_Nyq = 90/ np.mean(t_arr[1:]) # deg per Gyr
f_Nyq_rad = (f_Nyq*u.deg/u.Gyr).to(u.rad*u.km/u.s/u.kpc)

cum_t_arr = np.cumsum(t_arr)

In [6]:
len(saarthak_catalogue)

40

In [7]:
r0 = np.array([1,1,1])/np.sqrt(3)
no_products = 0
good_rotation_dict = {}
count_good = 0
good_catalogue = []
for GrNr in tqdm(saarthak_catalogue):
    omega_file = saveDir + "GrNr_%d_snap_%d_99_patternSpeeds.npy"%(GrNr,startSnap)
    if exists(omega_file):
        Omega = np.load(saveDir+"GrNr_%d_snap_%d_99_patternSpeeds.npy"%(GrNr,startSnap),allow_pickle=True).item()

    ##########################################################################################################
        # Measure the analytic error
        axisRatios = np.load(loadDir+"GrNr_%d_snap_%d_99_axisRatios_full.npy"%(GrNr,startSnap),allow_pickle=True)
        Npart = np.load(loadDir+"GrNr_%d_snap_%d_99_Npart.npy"%(GrNr,startSnap))

        p_e = (1-2*axisRatios[:,0] + axisRatios[:,1])/(1-axisRatios[:,1])

        analytic_err = analytic_error([p_e,Npart],*errFitConsts)

        ##########################################################################################################
        # put a cut on plane chi2
        plane_chi2 = np.array(Omega['plane_chi2'][0])
        plane_chi2_cut = np.array(plane_chi2) <= 10

        ##########################################################################################################
        # put a cut on chi2
        chi2 = Omega['chi2']
        chi2_cut = np.array(chi2) <= 20

        ##########################################################################################################
        # put a cut on lower detection limit
        # measure the rotation durations
        durations = Omega['startStop']

        rotation_axes = np.array(Omega['Raxis'])
        O = np.array(Omega['Omega'])

        t = np.array([cum_t_arr[interval[1]]-cum_t_arr[interval[0]] for interval in durations])
        undefDuration = np.array([False if 0 in interval else True for interval in durations])

        # change in p/e over rotation duration
        delta_p_e = np.array([p_e[interval[1]]-p_e[interval[0]] for interval in durations])
        avg_p_e   = np.array([np.mean(p_e[interval[0]:interval[1]]) for interval in durations])

        # measure the uncertainty on a vector in the plane of rotation, for each 
        max_ang_err = []
        for epoch_i in range(len(rotation_axes)):
            vec_in_plane = np.cross(r0,rotation_axes[epoch_i])
            vec_in_plane = vec_in_plane/np.sqrt(np.sum(vec_in_plane**2))

            max_ang_err.append(np.max(abs(analytic_err[durations[epoch_i][0]:durations[epoch_i][1]].dot(vec_in_plane))))

        f_lower_bound = np.array(max_ang_err)*180/np.pi / t # in deg / Gyr
        #print(f_lower_bound * np.pi/180)

        lower_bound_cut = np.abs(O) > f_lower_bound # [True for sampleNum in range(len(chi2_cut))] #

        ##########################################################################################################
        # put a cut on duration, must be at least 4 snapshots
        duration_nSnaps = np.array([Omega['startStop'][i][1]-Omega['startStop'][i][0] for i in range(len(Omega['startStop']))])
        duration_time = np.array([cum_t_arr[Omega['startStop'][i][1]]-\
                                  cum_t_arr[Omega['startStop'][i][0]] for i in range(len(Omega['startStop']))])
        durations_less_4 = ~(duration_nSnaps<3)

        ##########################################################################################################
        # apply each cut, append any results that are still good detections

        cut_full = plane_chi2_cut & chi2_cut & lower_bound_cut & durations_less_4

        if any(cut_full):
            count_good += 1
            good_catalogue.append(GrNr)

        good_rotation_dict[GrNr] = {'Omega':np.array(Omega['Omega'])[cut_full],
                                    'startStop':np.array(Omega['startStop'])[cut_full],
                                    'Raxis':np.array(Omega['Raxis'])[cut_full]}
    else:
        no_products += 1
np.save('Saarthak_catalogue_good_FR_selected.npy',good_rotation_dict,allow_pickle=True)
count_good

100%|██████████| 40/40 [00:00<00:00, 347.37it/s]


17

In [8]:
no_products

8

In [9]:
good_rotation_dict

{516: {'Omega': array([43.52399784, 29.94108567, -7.38514489]),
  'startStop': array([[ 0,  6],
         [ 8, 18],
         [18, 22]]),
  'Raxis': array([[-0.0291765 ,  0.04020516,  0.99876538],
         [ 0.04711703,  0.07473954,  0.99608935],
         [ 0.08210518,  0.02266469,  0.99636592]])},
 601: {'Omega': array([], dtype=float64),
  'startStop': array([], shape=(0, 2), dtype=int64),
  'Raxis': array([], shape=(0, 3), dtype=float64)},
 587: {'Omega': array([], dtype=float64),
  'startStop': array([], shape=(0, 2), dtype=int64),
  'Raxis': array([], shape=(0, 3), dtype=float64)},
 772: {'Omega': array([19.21431948, -6.38598623, 14.67920299]),
  'startStop': array([[ 0, 15],
         [15, 20],
         [20, 24]]),
  'Raxis': array([[ 0.02722498,  0.03697612,  0.99894523],
         [ 0.25869605,  0.34095664,  0.90378367],
         [-0.09636924,  0.14106911,  0.98529817]])},
 920: {'Omega': array([ 3.29551823, 10.53819242]),
  'startStop': array([[ 0, 10],
         [19, 22]]),
  'Rax